In [10]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow_hub as hub
import tensorflow_datasets as tfds

In [11]:
print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

Version:  2.0.0-beta1
Eager mode:  True
Hub version:  0.5.0
GPU is NOT AVAILABLE


**IMDB Dataset**
from https://github.com/tensorflow/datasets
ref https://github.com/tensorflow/datasets/blob/master/docs/datasets.md#imdb_reviews

Large Movie Review Dataset. This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.

In [12]:
train_validation_split = tfds.Split.TRAIN.subsplit([6, 4])

(train_data, validation_data), test_data = tfds.load(
    name="imdb_reviews", 
    split=(train_validation_split, tfds.Split.TEST),
    as_supervised=True
)

In [16]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
# train_examples_batch

In [14]:
train_labels_batch #0=negative 1=positive

<tf.Tensor: id=446, shape=(2,), dtype=int64, numpy=array([1, 1])>

use **pre-trained text embedding model**

https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1

In [17]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: id=650, shape=(3, 20), dtype=float32, numpy=
array([[ 3.9819887 , -4.4838037 ,  5.177359  , -2.3643482 , -3.2938678 ,
        -3.5364532 , -2.4786978 ,  2.5525482 ,  6.688532  , -2.3076782 ,
        -1.9807833 ,  1.1315885 , -3.0339816 , -0.7604128 , -5.743445  ,
         3.4242578 ,  4.790099  , -4.03061   , -5.992149  , -1.7297493 ],
       [ 3.4232912 , -4.230874  ,  4.1488533 , -0.29553518, -6.802391  ,
        -2.5163853 , -4.4002395 ,  1.905792  ,  4.7512794 , -0.40538004,
        -4.3401685 ,  1.0361497 ,  0.9744097 ,  0.71507156, -6.2657013 ,
         0.16533905,  4.560262  , -1.3106939 , -3.1121316 , -2.1338716 ],
       [ 3.8508697 , -5.003031  ,  4.8700504 , -0.04324996, -5.893603  ,
        -5.2983093 , -4.004676  ,  4.1236343 ,  6.267754  ,  0.11632943,
        -3.5934832 ,  0.8023905 ,  0.56146765,  0.9192484 , -7.3066816 ,
         2.8202746 ,  6.2000837 , -3.5709393 , -4.564525  , -2.305622  ]],
      dtype=float32)>

In [18]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [20]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [21]:
history = model.fit(
    train_data.shuffle(10000).batch(512),
    epochs=20,
    validation_data=validation_data.batch(512),
    verbose=1
)

Epoch 1/20


W0712 04:55:53.552644 139714349422336 deprecation.py:323] From /usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


30/30 [==============================] - 12s 390ms/step - loss: 0.7037 - accuracy: 0.6017 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/20
30/30 [==============================] - 10s 328ms/step - loss: 0.6193 - accuracy: 0.6603 - val_loss: 0.5974 - val_accuracy: 0.6844
Epoch 3/20
30/30 [==============================] - 10s 325ms/step - loss: 0.5852 - accuracy: 0.6978 - val_loss: 0.5714 - val_accuracy: 0.7128
Epoch 4/20
30/30 [==============================] - 10s 332ms/step - loss: 0.5569 - accuracy: 0.7235 - val_loss: 0.5467 - val_accuracy: 0.7334
Epoch 5/20
30/30 [==============================] - 10s 325ms/step - loss: 0.5275 - accuracy: 0.7459 - val_loss: 0.5212 - val_accuracy: 0.7545
Epoch 6/20
30/30 [==============================] - 10s 329ms/step - loss: 0.4963 - accuracy: 0.7729 - val_loss: 0.4941 - val_accuracy: 0.7723
Epoch 7/20
30/30 [==============================] - 10s 326ms/step - loss: 0.4636 - accuracy: 0.7968 - val_loss: 0.4662 - val_accuracy: 0.7904
Ep

In [22]:
results = model.evaluate(test_data.batch(512), verbose=0)
for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

loss: 0.319
accuracy: 0.864
